In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 120.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 80.6 MB/s eta 0:00:00


In [2]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 11.2 MB/s eta 0:00:00


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image
import numpy as np

# Define the custom dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, label_file, transform=None):
        self.root_dir = root_dir
        self.label_file = label_file
        self.transform = transform

        # Read the labels from the Excel file
        self.labels = pd.read_excel(label_file)

        # Get the list of JPEG files in the root directory
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpeg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get the image file name
        image_file = self.image_files[idx]

        # Read the image from the file
        image = Image.open(os.path.join(self.root_dir, image_file)).convert('RGB')

        # Apply the transform to the image
        if self.transform:
            image = self.transform(image)

        # Get the label for the image
        label =torch.tensor(self.labels.values).T[idx+1]/100

        # Return the image and the label
        return image.float(), label.float()



# Define the ResNet50 model
resnet50 = torchvision.models.resnet50(pretrained=True)

# Freeze the ResNet50 model parameters
for param in resnet50.parameters():
    param.requires_grad = False

# Add a custom fully connected layer on top of the ResNet50 model
resnet50.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=500),
     nn.Linear(in_features=500, out_features=50),
    nn.Linear(in_features=50, out_features=20),
    nn.Linear(in_features=20, out_features=6),
    nn.Sigmoid()
)
model = resnet50
# Move the model to GPU if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model= nn.DataParallel(model,device_ids=[0,1])


# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.001)

# Define the data transformations
transform = transforms.Compose([
    #transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create the custom dataset
dataset = CustomDataset(root_dir='/workspace/low_fine/',
                        label_file='/workspace/low_fine/select_low_fine.xlsx', transform=transform)

# Split the dataset into training, validation, and test sets
train_size = int(0.9 * len(dataset))
val_size = int(0.05 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)


#Train the model
num_epochs = 70
for epoch in range(num_epochs):
    # Train the model for one epoch
    resnet50.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move the inputs and targets to the GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Update the weights
        optimizer.step()

        # Print the loss
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {loss.item()}')

    # Evaluate the model on the validation set
    resnet50.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # Move the inputs and targets to the GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = resnet50(inputs)

            # Compute the loss
            val_loss += criterion(outputs, targets).item()

    # Print the validation loss
    val_loss /= len(val_loader)
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss}')

def mape(y_true, y_pred):
    # Avoid division by zero
    y_true = torch.clamp(y_true, min=1e-8)
    # Compute the absolute percentage error
    ape = torch.abs((y_true - y_pred) / y_true)
    # Return the mean over all predictions
    return torch.mean(ape)

# Test the model on the test set
resnet50.eval()
with torch.no_grad():
    test_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # Move the inputs and targets to the GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = resnet50(inputs)

        # Compute the loss
        test_loss += criterion(outputs, targets).item()

    # Print the test loss
    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss}')
torch.save(model.state_dict(), 'resnet_mlp_model.pt')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Batch: 1, Loss: 0.17776905000209808
Epoch: 1, Validation Loss: 0.012542091310024261
Epoch: 2, Batch: 1, Loss: 0.03256154805421829
Epoch: 2, Validation Loss: 0.02199721336364746
Epoch: 3, Batch: 1, Loss: 0.04129542410373688
Epoch: 3, Validation Loss: 0.039320189505815506
Epoch: 4, Batch: 1, Loss: 0.03632071241736412
Epoch: 4, Validation Loss: 0.004913962446153164
Epoch: 5, Batch: 1, Loss: 0.05836314707994461
Epoch: 5, Validation Loss: 0.01823289692401886
Epoch: 6, Batch: 1, Loss: 0.02373502030968666
Epoch: 6, Validation Loss: 0.03409358859062195
Epoch: 7, Batch: 1, Loss: 0.04893311485648155
Epoch: 7, Validation Loss: 0.03554613143205643
Epoch: 8, Batch: 1, Loss: 0.03209806606173515
Epoch: 8, Validation Loss: 0.0038462996017187834
Epoch: 9, Batch: 1, Loss: 0.01797446236014366
Epoch: 9, Validation Loss: 0.02861448936164379
Epoch: 10, Batch: 1, Loss: 0.026404354721307755
Epoch: 10, Validation Loss: 0.036693040281534195
Epoch: 11, Batch: 1, Loss: 0.016026761382818222
Epoch: 11, Va

In [ ]:
torch.save(model.state_dict(), 'resnet_mlp_model.pt')

In [4]:
def mape(y_true, y_pred):
    # Avoid division by zero
    y_true = torch.clamp(y_true, min=1e-8)
    # Compute the absolute percentage error
    ape = torch.abs((y_true - y_pred) / y_true)*100
    # Return the mean over all predictions
    return torch.mean(ape)

with torch.no_grad():
    test_loss = 0.0
    test_mape = 0.0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # Move the inputs and targets to the GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = resnet50(inputs)

        # Compute the loss
        test_loss += criterion(outputs, targets).item()
        # Compute the MAPE
        test_mape += mape(targets, outputs).item()

    # Print the test loss and MAPE
    test_loss /= len(test_loader)
    test_mape /= len(test_loader)
    print(f'Test Loss: {test_loss}')
    print(f'Test MAPE: {test_mape}')

Test Loss: 0.028416287153959274
Test MAPE: 34.75172805786133
